# Security Tool Tampering (AV/EDR Disabled)

**Security services stopped/disabled: possible defense evasion or unauthorized admin activity.**

This notebook is a **ready-to-use SOC playbook** with:

- Triage checklist
- Splunk SPL queries
- Microsoft Sentinel KQL queries
- Elastic KQL/EQL concepts
- Containment & response commands (PowerShell / Windows / MDE)

> Replace placeholders like `<USER>`, `<HOST>`, `<DEVICE_ID>`, `<TENANT_DOMAIN>` before running.


## Triage Checklist

**Collect:** user/account, host/device, source IP, time window, impacted resources, and any correlated alerts.

**Validate:** baseline/allowlists (VPN egress, jump hosts, admin tools) before escalating.

**Preserve:** keep a copy of key logs and process trees before remediation when possible.

Focus: **which service**, **who stopped it**, **parent process**, **other suspicious activity nearby**.


## Investigation Queries

Look for service stop/disable events and correlate with process creation.


In [ ]:
// KQL
DeviceEvents
| where TimeGenerated > ago(30d)
| where ActionType has_any ('ServiceStopped','ServiceDisabled')
| where AdditionalFields has_any ('WinDefend','Sense','WdNisSvc','Sysmon')
| project TimeGenerated, DeviceName, ActionType, AdditionalFields, InitiatingProcessAccountName


In [ ]:
# Splunk SPL
index=endpoint (EventCode=7036 OR EventCode=7040) (ServiceName=WinDefend OR ServiceName=Sense OR ServiceName=WdNisSvc)
| table _time host ServiceName EventCode Message


In [ ]:
# PowerShell
## Check service status locally
Get-Service WinDefend, Sense, WdNisSvc -ErrorAction SilentlyContinue | Format-Table Name, Status, StartType


## Containment & Response

Re-enable defenses (if legitimate), isolate host if correlated to malware, and investigate privilege misuse.


In [ ]:
# PowerShell
## Attempt to start Defender service (if allowed)
Start-Service WinDefend

## Ensure service start type (if policy allows)
Set-Service WinDefend -StartupType Automatic


In [ ]:
# PowerShell
## Consider enabling Tamper Protection (tenant setting) and alerting on attempts.
